In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO

url = 'https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/credential_access/host/empire_dcsync_dcerpc_drsuapi_DsGetNCChanges.zip'
zipFileRequest = requests.get(url)
zipFile = ZipFile(BytesIO(zipFileRequest.content))
datasetJSONPath = zipFile.extract(zipFile.namelist()[0])

In [2]:
import pandas as pd
from pandas.io import json

df = json.read_json(path_or_buf=datasetJSONPath, lines=True)

C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
(
df[['@timestamp','Hostname','SubjectUserName','SubjectLogonId']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4662)
    & (df['AccessMask'] == '0x100')
    & (
        (df['Properties'].str.contains('.*1131f6aa-9c07-11d1-f79f-00c04fc2dcd2.*', regex=True))
        | (df['Properties'].str.contains('.*1131f6ad-9c07-11d1-f79f-00c04fc2dcd2.*', regex=True))
        | (df['Properties'].str.contains('.*89e95b76-444d-4c62-991a-0facbeda640c.*', regex=True))
    )
    & (~df['SubjectUserName'].str.endswith('.*$', na=False))
]
)

,@timestamp,Hostname,SubjectUserName,SubjectLogonId
5125,2020-09-21T22:59:14.904Z,MORDORDC.theshire.local,pgustavo,0x69099c7
5126,2020-09-21T22:59:14.904Z,MORDORDC.theshire.local,pgustavo,0x69099c7
5127,2020-09-21T22:59:14.904Z,MORDORDC.theshire.local,pgustavo,0x69099c7


In [10]:
adObjectAccessDf = (
# (
df[['@timestamp','Hostname','SubjectUserName','SubjectLogonId']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4662)
    & (df['AccessMask'] == '0x100')
    & (
        (df['Properties'].str.contains('.*1131f6aa-9c07-11d1-f79f-00c04fc2dcd2.*', regex=True))
        | (df['Properties'].str.contains('.*1131f6ad-9c07-11d1-f79f-00c04fc2dcd2.*', regex=True))
        | (df['Properties'].str.contains('.*89e95b76-444d-4c62-991a-0facbeda640c.*', regex=True))
    )
    & (~df['SubjectUserName'].str.endswith('.*$', na=False))
]
)

networkLogonDf = (
# (
df[['@timestamp','Hostname','TargetUserName','TargetLogonId','IpAddress']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4624)
    & (df['LogonType'] == 3)
    & (~df['SubjectUserName'].str.endswith('.*$', na=False))
]
)

(
pd.merge(adObjectAccessDf, networkLogonDf,
    left_on = 'SubjectLogonId', right_on = 'TargetLogonId', how = 'inner')
)

,@timestamp_x,Hostname_x,SubjectUserName,SubjectLogonId,@timestamp_y,Hostname_y,TargetUserName,TargetLogonId,IpAddress
0,2020-09-21T22:59:14.904Z,MORDORDC.theshire.local,pgustavo,0x69099c7,2020-09-21T22:59:14.904Z,MORDORDC.theshire.local,pgustavo,0x69099c7,172.18.39.5
1,2020-09-21T22:59:14.904Z,MORDORDC.theshire.local,pgustavo,0x69099c7,2020-09-21T22:59:14.904Z,MORDORDC.theshire.local,pgustavo,0x69099c7,172.18.39.5
2,2020-09-21T22:59:14.904Z,MORDORDC.theshire.local,pgustavo,0x69099c7,2020-09-21T22:59:14.904Z,MORDORDC.theshire.local,pgustavo,0x69099c7,172.18.39.5
